In [21]:
import sys
from pathlib import Path
sys.path.append(str(Path("../src").absolute().resolve()))

from model import *
from data import VAEDataset
from experiment import VAEXperiment
from callbacks import *
from util import *

import torch.backends.cudnn as cudnn
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [17]:
config = load_config("../configs/vae.yaml")

In [24]:
config['data_params']['data_path'] = "../Data/"

In [22]:
tb_logger =  TensorBoardLogger(save_dir=config['logging_params']['save_dir'],
                                   name=config['model_params']['name'],
                                   log_graph=False,
                                   default_hp_metric=False,
                                   flush_secs=5,)

In [25]:
model = VAE(**config['model_params'])
experiment = VAEXperiment(model, config['exp_params'])
data = VAEDataset(**config["data_params"], pin_memory=len(config['trainer_params']['gpus']) != 0)

In [43]:
data.setup()
runner = Trainer(logger=tb_logger,
                 callbacks=[
                     CPUMonitor(),
                     LearningRateMonitor("epoch"),
                     ModelCheckpoint(save_top_k=2, 
                                     dirpath =os.path.join(tb_logger.log_dir , "checkpoints"), 
                                     monitor= "val_loss",
                                     save_last= True),
                 ],
                 log_every_n_steps=1,
                 accelerator="auto",
                 devices=1 if torch.cuda.is_available() else None,
                 **config['trainer_params'])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [47]:
runner.tuner.lr_find(experiment, datamodule=data)

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

LR finder stopped early after 87 steps due to diverging loss.
Restoring states from the checkpoint path at C:\Users\qwert\OneDrive\桌面\Research\NTUH\src\dl_training\notebooks\.lr_find_81374e69-3711-46d7-b7b7-42706ccd66dc.ckpt


In [28]:
runner.fit(experiment, datamodule=data)


  | Name  | Type | Params
-------------------------------
0 | model | VAE  | 2.1 M 
-------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.327     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [45]:
val_result = runner.test(experiment, datamodule=data, verbose=True)

MisconfigurationException: No `test_step()` method defined to run `Trainer.test`.

In [46]:
tb_logger.experiment.add_graph()